In [6]:
import jieba
import logging

def jiebaProcess():

    logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

    jieba.load_userdict("userDict.txt")
    # load stopwords set
    stopword_set = set()
    with open('stopwords.txt','r', encoding='utf-8') as stopwords:
        for stopword in stopwords:
            stopword_set.add(stopword.strip('\n'))

    output = open('san_seg.txt', 'w', encoding='utf-8')
    with open('san.txt', 'r', encoding='utf-8') as content :
        for texts_num, line in enumerate(content):
            line = line.strip('\n')
            words = jieba.cut(line, cut_all=False)
            for word in words:
                if word not in stopword_set:
                    output.write(word + ' ')
            output.write('\n')

            if (texts_num + 1) % 10000 == 0:
                logging.info("已完成前 %d 行的斷詞" % (texts_num + 1))
    output.close()

In [7]:
jiebaProcess()

In [8]:
from gensim.models import word2vec

def train():

    logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)
    sentences = word2vec.LineSentence("san_seg.txt")
    model = word2vec.Word2Vec(sentences, size=250)

    #保存模型，供日後使用
    model.save("word2vec.model")

    #模型讀取方式
    # model = word2vec.Word2Vec.load("your_model_name")

train()

2020-03-26 18:18:19,312 : WARNING : consider setting layer size to a multiple of 4 for greater performance
2020-03-26 18:18:19,315 : INFO : collecting all words and their counts
2020-03-26 18:18:19,318 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2020-03-26 18:18:19,496 : INFO : collected 64381 word types from a corpus of 345185 raw words and 8105 sentences
2020-03-26 18:18:19,497 : INFO : Loading a fresh vocabulary
2020-03-26 18:18:19,548 : INFO : effective_min_count=5 retains 10464 unique words (16% of original 64381, drops 53917)
2020-03-26 18:18:19,549 : INFO : effective_min_count=5 leaves 263247 word corpus (76% of original 345185, drops 81938)
2020-03-26 18:18:19,586 : INFO : deleting the raw counts dictionary of 64381 items
2020-03-26 18:18:19,588 : INFO : sample=0.001 downsamples 18 most-common words
2020-03-26 18:18:19,589 : INFO : downsampling leaves estimated 232816 word corpus (88.4% of prior 263247)
2020-03-26 18:18:19,610 : INFO : estimated r

In [9]:
from gensim.models import word2vec
from gensim import models
import logging

def demo():
	logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)
	model = models.Word2Vec.load('word2vec.model')

	print("提供3种测试模式\n")
	print("输入一个词，则去寻找前10个相似的词")
	print("输入两个词，计算两个词的相似度")
	print("输入三个词，则进行词类推")

	while True:
		try:
			query = input()
			q_list = query.split()

			if len(q_list) == 1:
				print("相似词 10 排序")
				res = model.most_similar(q_list[0],topn = 10)
				for item in res:
					print(item[0]+","+str(item[1]))

			elif len(q_list) == 2:
				print("计算 Cosine 相似度")
				res = model.similarity(q_list[0],q_list[1])
				print(res)
			else:
				print("%s之于%s，如%s之于" % (q_list[0],q_list[2],q_list[1]))
				res = model.most_similar([q_list[0],q_list[1]], [q_list[2]], topn= 10)
				for item in res:
					print(item[0]+","+str(item[1]))
			print("----------------------------")
		except Exception as e:
			print(repr(e))

In [ ]:
demo()

2020-03-26 18:18:23,996 : INFO : loading Word2Vec object from word2vec.model
2020-03-26 18:18:24,180 : INFO : loading wv recursively from word2vec.model.wv.* with mmap=None
2020-03-26 18:18:24,181 : INFO : setting ignored attribute vectors_norm to None
2020-03-26 18:18:24,182 : INFO : loading vocabulary recursively from word2vec.model.vocabulary.* with mmap=None
2020-03-26 18:18:24,182 : INFO : loading trainables recursively from word2vec.model.trainables.* with mmap=None
2020-03-26 18:18:24,183 : INFO : setting ignored attribute cum_table to None
2020-03-26 18:18:24,183 : INFO : loaded word2vec.model


提供3种测试模式

输入一个词，则去寻找前10个相似的词
输入两个词，计算两个词的相似度
输入三个词，则进行词类推


 刘备 的卢


/Users/huqianghui/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:27: DeprecationWarning: Call to deprecated `similarity` (Method will be removed in 4.0.0, use self.wv.similarity() instead).


计算 Cosine 相似度
KeyError("word '的卢' not in vocabulary")
